# Creating Set of Augmentation Slices

In [ ]:
### CREATING AUGMENTATION FILES ##
######################################
import os
import numpy as np
import cv2
import albumentations as A

# Paths
original_images_path = 'data/slices/training_slices/img'
original_masks_path = 'data/slices/training_slices/mask'
augmented_images_path = 'data/slices/training_slices/augimg'
augmented_masks_path = 'data/slices/training_slices/augmask'
os.makedirs(augmented_images_path, exist_ok=True)
os.makedirs(augmented_masks_path, exist_ok=True)

# Augmentation pipeline
augmentations = A.Compose([A.HorizontalFlip(p=0.5),
                           A.VerticalFlip(p=0.5),
                           A.RandomBrightnessContrast(p=0.2)])

# Load and augment images
image_filenames = sorted(os.listdir(original_images_path))
mask_filenames = sorted(os.listdir(original_masks_path))
assert len(image_filenames) == len(mask_filenames), "Mismatch in number of image and mask files"

for img_file, mask_file in zip(image_filenames, mask_filenames):
    img = cv2.imread(os.path.join(original_images_path, img_file), cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(os.path.join(original_masks_path, mask_file), cv2.IMREAD_GRAYSCALE)
    
    # Convert images to float32 numpy arrays
    img = img.astype(np.float32) / 255.0  # Normalize images to [0, 1]
    mask = mask.astype(np.float32) / 255.0  # Normalize masks to [0, 1]
    
    for i in range(5):  # Generate 5 augmented versions of each image
        augmented = augmentations(image=img, mask=mask)
        aug_img, aug_mask = augmented['image'], augmented['mask']
        aug_img_filename = f'aug_{i}_{img_file}'
        aug_mask_filename = f'aug_{i}_{mask_file}'
        
        # Convert images back to uint8 for saving
        aug_img = (aug_img * 255).astype(np.uint8)
        aug_mask = (aug_mask * 255).astype(np.uint8)
        cv2.imwrite(os.path.join(augmented_images_path, aug_img_filename), aug_img)
        cv2.imwrite(os.path.join(augmented_masks_path, aug_mask_filename), aug_mask)

print('Augmentation complete. Check the augmented images and masks at the specified paths.')
